# Purpose and use

This Jupyter Notebook allows for the time series and general analysis of the scraped Wikipedia revisions.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

from config import *
from utils.utils import *
from utils.plot_graphs import *
## importing common data
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gc

import numpy as np
import pandas as pd
from PIL import Image

# plot
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# annotations
from highlight_text import fig_text, ax_text
from matplotlib.patches import FancyArrowPatch

import re

#import go
from plotly import graph_objects as go
import mwparserfromhell
from datetime import datetime, timedelta
import pytz


c:/Users/limyi/OneDrive/Documents/oxford/Fundamentals for Social Data Science/group_project/wiki/scripts


In [2]:
#clean up memory
gc.collect()

0

In [3]:
#writing to reqs
!pip3 freeze > requirements.txt

# Read in Data

To facilitate reading in the large amount of data, we'll write a helper function to read the feather dfs in chunks.

In [4]:
DATA_DIR

'c:/Users/limyi/OneDrive/Documents/oxford/Fundamentals for Social Data Science/group_project/wiki/data'

In [5]:
#declare file paths
file_path_Taylor = DATA_DIR +  "/DataFrames/Taylor_Swift.feather"
file_path_Kanye =  DATA_DIR +  "/DataFrames/Kanye_West.feather"

In [6]:
#reading in data
ts_df = read_feather_data(file_path=file_path_Taylor) # Taylor Swift
ky_df = read_feather_data(file_path=file_path_Kanye) # Kanye West

In [7]:
#clean up memory
gc.collect()

0

In [8]:
#check
ts_df.head()

,revision_id,timestamp,username,userid,comment,text_length,text,year,month,day
0,1253532621,2024-10-26 14:03:32+00:00,Jessintime,44776426,Undid revision [[Special:Diff/1253531462|12535...,359306,{{Short description|American singer-songwriter...,Taylor_Swift,2024,10
1,1253531462,2024-10-26 13:55:46+00:00,Chuckles987,48629687,Updated short description,359295,{{Short description|American singer (born 1989...,Taylor_Swift,2024,10
2,1253458894,2024-10-26 02:49:42+00:00,NegativeMP1,45135659,Restored revision 1253426413 by [[Special:Cont...,359306,{{Short description|American singer-songwriter...,Taylor_Swift,2024,10
3,1253458276,2024-10-26 02:44:43+00:00,Hammsster,47189514,None,359462,{{Short description|American singer-songwriter...,Taylor_Swift,2024,10
4,1253457996,2024-10-26 02:42:36+00:00,Hammsster,47189514,ancestry,359577,{{Short description|American singer-songwriter...,Taylor_Swift,2024,10


In [9]:
#check
ky_df.head()

,revision_id,timestamp,username,userid,comment,text_length,text,year,month,day
0,1253370019,2024-10-25 16:44:50+00:00,BigChungusOnVinyl,35552696,/* Discography */,291490,{{Short description|American rapper and produc...,Kanye_West,2024,10
1,1253369815,2024-10-25 16:43:20+00:00,ULPS,44938552,/* 2023–present: Vultures trilogy and Bully */...,291472,{{Short description|American rapper and produc...,Kanye_West,2024,10
2,1253368276,2024-10-25 16:33:36+00:00,BigChungusOnVinyl,35552696,/* Discography */,291440,{{Short description|American rapper and produc...,Kanye_West,2024,10
3,1253106242,2024-10-24 11:18:59+00:00,Thedogishere931,48083651,/* Discography */,291470,{{Short description|American rapper and produc...,Kanye_West,2024,10
4,1252589484,2024-10-22 01:11:08+00:00,Speakfor23,46861074,/* Other relationships */,291440,{{Short description|American rapper and produc...,Kanye_West,2024,10


# Plot Timeseries

In [10]:
ky_df['timestamp'] = pd.to_datetime(ky_df['timestamp'])
ts_df['timestamp'] = pd.to_datetime(ts_df['timestamp'])

In [11]:
plot_timeseries(data = ky_df, x ='timestamp', y='revision_id', 
                title = 'Wikipedia Page Revisions Over Time -  Kanye West', x_title = 'Date', 
                y_title= 'Number of Revisions', file_path = OUTPUT_DIR + '/KW_revisions.html')

In [12]:
plot_timeseries(data = ts_df, x ='timestamp', y='revision_id', 
                title = 'Wikipedia Page Revisions Over Time - Taylor Swift', x_title = 'Date', 
                y_title= 'Number of Revisions', file_path = OUTPUT_DIR + '/TS_revisions.html')

In [13]:
ky_df.head()

,revision_id,username,userid,comment,text_length,text,year,month,day
timestamp,,,,,,,,,
2024-10-25 16:44:50+00:00,1253370019,BigChungusOnVinyl,35552696,/* Discography */,291490,{{Short description|American rapper and produc...,Kanye_West,2024,10
2024-10-25 16:43:20+00:00,1253369815,ULPS,44938552,/* 2023–present: Vultures trilogy and Bully */...,291472,{{Short description|American rapper and produc...,Kanye_West,2024,10
2024-10-25 16:33:36+00:00,1253368276,BigChungusOnVinyl,35552696,/* Discography */,291440,{{Short description|American rapper and produc...,Kanye_West,2024,10
2024-10-24 11:18:59+00:00,1253106242,Thedogishere931,48083651,/* Discography */,291470,{{Short description|American rapper and produc...,Kanye_West,2024,10
2024-10-22 01:11:08+00:00,1252589484,Speakfor23,46861074,/* Other relationships */,291440,{{Short description|American rapper and produc...,Kanye_West,2024,10


In [14]:
#prep for wordcloud KY
ky_text = ky_df.iloc[0]['text']
ky_content = preprocess(ky_text)

#prep for wordcloud TS
ts_text = ts_df.iloc[0]['text']
ts_content = preprocess(ts_text)

# save image to png
plot_wordcloud(ts_content, file_path = OUTPUT_DIR + '/TS_wordcloud.png')
plot_wordcloud(ky_content, file_path = OUTPUT_DIR + '/KY_wordcloud.png')

In [15]:
# Annotations
annotations = [
    {"x":"2022-12", "y": 540, "text": "Dec 2022: Antisemitic Remarks"},
    {"x": "2009-09", "y": 166, "text":  "Sep 2009: VMAs"}
]

plot_timeseries_with_annotations(
    data=ky_df,
    x="timestamp",
    y="revision_id",
    title="Editing History of <b>Kanye West</b>'s Wikipedia Pag",
    x_title="Date",
    y_title="Number of Wikipedia Page Edits",
    annotations=annotations,
    wordcloud_file_path=OUTPUT_DIR + '/KY_wordcloud.png'
)

In [16]:
# Annotations
annotations = [
    {"x":"2012-05", "y": 662, "text": "May 2012: Billboard Music Awards"},
    {"x": "2009-09", "y": 211, "text":  "Sep 2009: VMAs"}
]

plot_timeseries_with_annotations(
    data=ts_df,
    x="timestamp",
    y="revision_id",
    title="Editing History of <b>Taylor Swift</b>'s Wikipedia Pag",
    x_title="Date",
    y_title="Number of Wikipedia Page Edits",
    annotations=annotations,
    wordcloud_file_path=OUTPUT_DIR + '/TS_wordcloud.png'
)

# Editors analysis

In [17]:
#reset index
ky_df = ky_df.reset_index()
ts_df = ts_df.reset_index()

In [18]:
#value counts on userid
ky_df['userid'].value_counts()

userid
16899771    492
19493411    242
4293477     240
6661051     199
19507760    107
           ... 
2572687       1
2795635       1
43123791      1
48262630      1
4319153       1
Name: count, Length: 2886, dtype: int64

In [19]:
#value counts on userid
ts_df['userid'].value_counts()

userid
6138283     2375
27173459    1935
19895717    1219
35372709     391
19269270     324
            ... 
271372         1
1792859        1
1354450        1
2677754        1
2842348        1
Name: count, Length: 3015, dtype: int64

In [20]:
#comparing editors
ky_editors = ky_df['userid'].unique()
ts_editors = ts_df['userid'].unique()
common_editors = set(ky_editors).intersection(ts_editors)
print(common_editors)

{'14264656', '46913010', '1574439', '5203934', '33299132', '29189256', '15996738', '7667333', '14508071', '11181023', '285130', '2935937', '41275174', '409043', '27173459', '4007668', '6085301', '12089300', '1234927', '9806113', '4928500', '256455', '42425010', '16489436', '2880605', '41929512', '8372814', '13286072', '6261166', '929196', '1902946', '16710', '7662999', '28568099', '18934407', '1670386', '41474409', '24965548', '205121', '3174456', '20320531', '44120587', '19912518', '9505858', '37523058', '27823944', '236191', '37528485', '21748642', '19269270', '21145068', '33609231', '7431540', '159678', '29919047', '42076370', '27738727', '7713040', '6593610', '4871659', '1286970', '118799', '1416331', '29963873', '1244619', '5202324', '20588153', '4175835', '31880358', '14954744', '33511315', '35667443', '19130652', '264323', '40327909', '237572', '11292982', '19986047', '3492060', '196446', '6436929', '8168717', '25082147', '33163851', '29269934', '2530149', '12323565', '9107368',

# Analysis VMA

In [21]:
VMA_date = datetime.strptime("2009-09-13", "%Y-%m-%d").replace(tzinfo=pytz.UTC)

In [22]:
date_one_week_before = VMA_date - timedelta(days=7)
date_one_week_after = VMA_date + timedelta(days=7)

In [23]:
ts_df['timestamp'] = pd.to_datetime(ts_df['timestamp'])
ky_df['timestamp'] = pd.to_datetime(ky_df['timestamp'])

In [24]:
ts_df_VMA = ts_df[(ts_df['timestamp'] > date_one_week_before) & (ts_df['timestamp'] < date_one_week_after)]
ky_df_VMA = ky_df[(ky_df['timestamp'] > date_one_week_before) & (ky_df['timestamp'] < date_one_week_after)]

In [25]:
ts_df_VMA_after = ts_df[(ts_df['timestamp'] > VMA_date) & (ts_df['timestamp'] < date_one_week_after)]
ky_df_VMA_after = ky_df[(ky_df['timestamp'] > VMA_date) & (ky_df['timestamp'] < date_one_week_after)]

In [26]:
ts_before = ts_df[(ts_df['timestamp'] > date_one_week_before) & (ts_df['timestamp'] < VMA_date)].count()
ts_during = ts_df[(ts_df['timestamp'].dt.date == VMA_date.date())].count()
ts_after = ts_df[(ts_df['timestamp'] > VMA_date) & (ts_df['timestamp'] < date_one_week_after)].count()


In [27]:
ky_before = ky_df[(ky_df['timestamp'] > date_one_week_before) & (ky_df['timestamp'] < VMA_date)].count()
ky_during = ky_df[(ky_df['timestamp'].dt.date == VMA_date.date())].count() 
ky_after = ky_df[(ky_df['timestamp'] > VMA_date) & (ky_df['timestamp'] < date_one_week_after)].count()

In [28]:
ts_df_VMA_after['parsed_text'] = ts_df_VMA_after['text'].apply(lambda x: mwparserfromhell.parse(x).strip_code())
ky_df_VMA_after['parsed_text'] = ky_df_VMA_after['text'].apply(lambda x: mwparserfromhell.parse(x).strip_code())

C:\Users\limyi\AppData\Local\Temp\ipykernel_22628\1361206015.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\limyi\AppData\Local\Temp\ipykernel_22628\1361206015.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
from difflib import ndiff

def find_text_differences(text1, text2):
    diff = list(ndiff(text1.split(), text2.split()))
    added = [word[2:] for word in diff if word.startswith('+ ')]
    removed = [word[2:] for word in diff if word.startswith('- ')]
    unchanged = [word[2:] for word in diff if word.startswith('  ')]
    
    return {
        "added": added,
        "removed": removed
    }

In [30]:
def find_text_differences_2(text1, text2):
    if text1 is None or text2 is None:
        return {"added": [], "removed": []}
    return find_text_differences(text1, text2)

In [31]:
ts_df_VMA_after = ts_df_VMA_after.sort_values(by='timestamp')
ts_df_VMA_after['text_diff'] = ts_df_VMA_after['text'].shift().combine(ts_df_VMA_after['text'], find_text_differences_2)

ts_df_VMA_after.head()

,timestamp,revision_id,username,userid,comment,text_length,text,year,month,day,parsed_text,text_diff
16624,2009-09-13 00:33:49+00:00,313473831,Fedisking,1883650,/* Personal life */ Reference to Tyler Dean's ...,51847,{{Otheruses4|the musician|her self-titled debu...,Taylor_Swift,2009,09,"Taylor Alison Swift (born December 13, 1989) i...","{'added': [], 'removed': []}"
16623,2009-09-13 01:23:11+00:00,313481073,Cwateyou,9154284,/* Band */,51881,{{Otheruses4|the musician|her self-titled debu...,Taylor_Swift,2009,09,"Taylor Alison Swift (born December 13, 1989) i...","{'added': ['*', 'Charlotte', 'Ray&nbsp;–', '[[..."
16622,2009-09-13 01:24:21+00:00,313481277,Cwateyou,9154284,/* Band */,51883,{{Otheruses4|the musician|her self-titled debu...,Taylor_Swift,2009,09,"Taylor Alison Swift (born December 13, 1989) i...","{'added': ['Stripper', 'Haynes&nbsp;–'], 'remo..."
16621,2009-09-13 01:24:59+00:00,313481360,Cwateyou,9154284,/* Band */,51847,{{Otheruses4|the musician|her self-titled debu...,Taylor_Swift,2009,09,"Taylor Alison Swift (born December 13, 1989) i...","{'added': [], 'removed': ['*', 'Stripper', 'Ha..."
16620,2009-09-13 01:26:49+00:00,313481608,Cwateyou,9154284,None,51865,{{Otheruses4|the musician|her self-titled debu...,Taylor_Swift,2009,09,"Taylor Alison Swift (born December 13, 1989) i...","{'added': ['(band)|Gloriana]],', '[[Selena', '..."


In [32]:
ky_df_VMA_after = ky_df_VMA_after.sort_values(by='timestamp')
ky_df_VMA_after['text_diff'] = ky_df_VMA_after['text'].shift().combine(ky_df_VMA_after['text'], find_text_differences_2)
ky_df_VMA_after.head()

,timestamp,revision_id,username,userid,comment,text_length,text,year,month,day,parsed_text,text_diff
8214,2009-09-14 01:43:12+00:00,313708718,Ricky3374,10347019,None,57681,{{pp-semi|small=yes}}\n{{Infobox Musical artis...,Kanye_West,2009,09,"Kanye Omari West (; born June 8, 1977) is an A...","{'added': [], 'removed': []}"
8213,2009-09-14 01:43:35+00:00,313708801,E1foley,1800521,/* Awards and controversy (2006) */,58154,{{pp-semi|small=yes}}\n{{Infobox Musical artis...,Kanye_West,2009,09,"Kanye Omari West (; born June 8, 1977) is an A...","{'added': ['During', 'the', '2009', '[[MTV', '..."
8212,2009-09-14 01:47:28+00:00,313709562,CharleyHart,5875731,None,58496,{{pp-semi|small=yes}}\n{{Infobox Musical artis...,Kanye_West,2009,09,"Kanye Omari West (; born June 8, 1977) is a co...","{'added': ['a', 'controversial,', 'self-concer..."
8211,2009-09-14 01:48:51+00:00,313709840,Silly Demetrius,7341291,None,58515,{{pp-semi|small=yes}}\n{{Infobox Musical artis...,Kanye_West,2009,09,"Kanye Omari West (; born June 8, 1977) is a co...","{'added': ['[[singer]],', 'and', 'major', 'ass..."
8210,2009-09-14 01:49:00+00:00,313709865,Jdtink18,4473461,None,58661,{{pp-semi|small=yes}}\n{{Infobox Musical artis...,Kanye_West,2009,09,"Kanye Omari West (; born June 8, 1977) is a co...","{'added': ['Kanye', 'is', 'also', 'well-known'..."


In [33]:
# check if in added in column text_diff the word 'controversy' or 'controversial' gets mentioned (count)
print(f"Controversy mentions for KW: {ky_df_VMA_after['text_diff'].apply(lambda x: any(word in ['controversy', 'controversial','Controversy','Controversial'] for word in map(str.lower, x['added']))).sum()}")
print(f"Controversy mentions for TS: {ts_df_VMA_after['text_diff'].apply(lambda x: any(word in ['controversy', 'controversial','Controversy','Controversial'] for word in map(str.lower, x['added']))).sum()}")


Controversy mentions for KW: 13
Controversy mentions for TS: 1


In [34]:
ts_df_VMA_before = ts_df[(ts_df['timestamp'] > date_one_week_before) & (ts_df['timestamp'] < VMA_date)]
ky_df_VMA_before = ky_df[(ky_df['timestamp'] > date_one_week_before) & (ky_df['timestamp'] < VMA_date)]

In [35]:
ts_df_VMA_before = ts_df_VMA_before.sort_values(by='timestamp')
ky_df_VMA_before = ky_df_VMA_before.sort_values(by='timestamp')

In [36]:
ts_last_before = ts_df_VMA_before.iloc[-1]['text']
ky_last_before = ky_df_VMA_before.iloc[-1]['text']

In [37]:
print(f"Controversy mentions for TS BEFORE VMAs: {ts_last_before.lower().count('controversy') + ts_last_before.lower().count('controversial')}")
print(f"Controversy mentions for KY BEFORE VMAs: {ky_last_before.lower().count('controversy') + ky_last_before.lower().count('controversial')}")

Controversy mentions for TS BEFORE VMAs: 0
Controversy mentions for KY BEFORE VMAs: 3


In [38]:
ts_df_VMA_after = ts_df[(ts_df['timestamp'] > VMA_date) & (ts_df['timestamp'] < date_one_week_after)]
ky_df_VMA_after = ky_df[(ky_df['timestamp'] > VMA_date) & (ky_df['timestamp'] < date_one_week_after)]

In [39]:
ts_df_VMA_after['userid'].value_counts(normalize=True).head(10)


userid
4293477     0.364865
2197222     0.081081
10508097    0.060811
10106521    0.054054
9154284     0.033784
6436929     0.020270
4810503     0.020270
5627478     0.020270
9458515     0.013514
9739013     0.013514
Name: proportion, dtype: float64

In [40]:
ky_df_VMA_after['userid'].value_counts(normalize=True).head(10)

userid
4293477    0.190789
2084988    0.085526
911178     0.046053
6436929    0.039474
4322169    0.026316
42788      0.026316
1800521    0.026316
2442447    0.026316
7581206    0.019737
1481857    0.019737
Name: proportion, dtype: float64

## Common Editors

In [41]:
#check for common editors
ky_editors_VMA = ky_df_VMA['userid'].unique()
ts_editors_VMA = ts_df_VMA['userid'].unique()

common_editors_VMA = set(ky_editors_VMA).intersection(ts_editors_VMA)

In [42]:
common_editors_VMA

{'10106521', '1800521', '4293477', '5627478', '6436929', '8675237'}

In [43]:
#most active editor
ky_df['userid'].value_counts()

userid
16899771    492
19493411    242
4293477     240
6661051     199
19507760    107
           ... 
2572687       1
2795635       1
43123791      1
48262630      1
4319153       1
Name: count, Length: 2886, dtype: int64

In [44]:
most_active_user_ts = ts_df['userid'].value_counts().idxmax()
ts_df['revision_id'].value_counts().sum()
#how many edits were made by most active user
ts_df[ts_df['userid'] == most_active_user_ts]['revision_id'].value_counts().sum()

np.int64(2375)

In [45]:
2375/ts_df['revision_id'].value_counts().sum()

np.float64(0.12305061913890472)

In [46]:
ky_df['revision_id'].value_counts().sum()

np.int64(9642)

# Taylor: Billboard

In [47]:
BB = datetime.strptime("2012-05-20", "%Y-%m-%d").replace(tzinfo=pytz.UTC)

In [48]:
date_one_week_before_BB = BB - timedelta(days=7)
date_one_week_after_BB = BB + timedelta(days=7)

In [49]:
ts_df_BB_after = ts_df[(ts_df['timestamp'] > BB) & (ts_df['timestamp'] < date_one_week_after_BB)]

In [50]:
ts_df_BB_after.head()

,timestamp,revision_id,username,userid,comment,text_length,text,year,month,day
14139,2012-05-26 23:25:56+00:00,494534513,Popeye191,6138283,/* Natural disaster funds */,149698,{{About|the musician|her self-titled debut alb...,Taylor_Swift,2012,05
14140,2012-05-26 23:21:32+00:00,494533999,Popeye191,6138283,None,149476,{{About|the musician|her self-titled debut alb...,Taylor_Swift,2012,05
14141,2012-05-26 23:13:46+00:00,494533134,Popeye191,6138283,/* Arts education */,149476,{{About|the musician|her self-titled debut alb...,Taylor_Swift,2012,05
14142,2012-05-26 23:11:30+00:00,494532875,Popeye191,6138283,/* Arts education */,149472,{{About|the musician|her self-titled debut alb...,Taylor_Swift,2012,05
14143,2012-05-26 23:06:25+00:00,494532257,Popeye191,6138283,/* Arts education */,149457,{{About|the musician|her self-titled debut alb...,Taylor_Swift,2012,05


In [51]:
#% of edits made by each userid
ts_df_BB_after['userid'].value_counts(normalize=True).head(10)

userid
6138283     0.672
16287261    0.112
12615911    0.064
16710       0.048
16849652    0.024
2530149     0.024
13096171    0.016
14457634    0.008
16300118    0.008
1423121     0.008
Name: proportion, dtype: float64

In [52]:
data = {
    'Date': ['One Week Before', 'One Week After'],
    'Kanye West Edits': [4, 152],
    'Taylor Swift Edits': [14, 148]
}
df = pd.DataFrame(data)

# Define inputs for the generic function
categories = 'Date'
series = ['Kanye West Edits', 'Taylor Swift Edits']
colors = ['#CCA677', '#5D4037']
title = 'Wikipedia Page Edits Before and After VMAs'
yaxis_title = 'Number of Edits'
percent_change_base = {
    'Kanye West Edits': 4,
    'Taylor Swift Edits': 14
}

# Create the plot
fig_edits = create_bar_plot(
    df=df,
    categories=categories,
    series=series,
    colors=colors,
    title=title,
    yaxis_title=yaxis_title,
    percent_change_base=percent_change_base
)

fig_edits.show()
